In [1]:
import sys
import plotly.express as px
import numpy as np
from gymnasium.wrappers import TimeLimit

sys.path.append('..')

from environment.mangoEnv import MangoEnv, MAX_AMOUNT, REDUCED
from environment.action import ACTIONS_NUM, AMOUNT_ORDERS_HIGH, Action, AMOUNT_REDUCED

In [2]:
render_mode = None
ext_mngo_price=0.1
init_mngo_pool_balance=1e6 / MAX_AMOUNT
init_usdc_pool_balance=1e5 / MAX_AMOUNT
init_treasury_size_usdc=150e6 / MAX_AMOUNT
usdc_user_balance_init=2e8 / MAX_AMOUNT
mngo_collateral_factor=1.5
arb_efficiency_factor=0.95

env = MangoEnv(
    render_mode=render_mode, 
    ext_mngo_price_mean=ext_mngo_price, 
    init_mngo_pool_balance=init_mngo_pool_balance,
    init_usdc_pool_balance=init_usdc_pool_balance,
    init_treasury_size_usdc=init_treasury_size_usdc,
    mngo_collateral_factor=mngo_collateral_factor,
    arb_efficiency_factor=arb_efficiency_factor,
    usdc_user_balance_init=usdc_user_balance_init
)

env = TimeLimit(env, max_episode_steps=100000)

In [3]:
env.reset()
total_reward = 0
done = False
step = 0
jackpot = 0

while not done:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if reward == 10.0:
        jackpot += 1
    step += 1
    done = terminated or truncated
print(f"Steps passed: {step}, total reward: {total_reward}, jackpots amount: {jackpot}")
print(f"Health_factor: {env.get_wrapper_attr('mango').get_user_health_factor()}")

Steps passed: 200, total reward: -24.8, jackpots amount: 0
Health_factor: -0.0007488148383279006


In [4]:
env.reset()
total_reward = 0
done = False

# for continuous amount
# smart_actions = (
#     (5, 18e6 / MAX_AMOUNT), # Buy MNGO on external market
#     (1, 18e6 / MAX_AMOUNT), # Put MNGO as collateral
#     (0, 1.0e7 / MAX_AMOUNT), # Swap USDC to MNGO
#     (2, 100e6 / MAX_AMOUNT), # Borrow
#     (4, 0.), # Just Relax
#     (4, 0.),
#     (4, 0.),
#     (4, 0.),
#     (4, 0.),
#     (4, 0.)
# )

if REDUCED:
    smart_actions = (
        3 * AMOUNT_REDUCED + 1,
        1 * AMOUNT_REDUCED + 1,
        0 * AMOUNT_REDUCED + 1,
        2 * AMOUNT_REDUCED + 2,
        13,
        13,
        13,
        13,
        13,
        13
    )
else:
    smart_actions = (
        Action.BUY_MNGO.value * AMOUNT_ORDERS_HIGH + 6,
        Action.PUT_COLLATERAL.value * AMOUNT_ORDERS_HIGH + 6,
        Action.SWAP_USDC_TO_MNGO.value * AMOUNT_ORDERS_HIGH + 6,
        Action.BORROW.value * AMOUNT_ORDERS_HIGH + 7,
        Action.NOP.value * AMOUNT_ORDERS_HIGH,
        Action.NOP.value * AMOUNT_ORDERS_HIGH,
        Action.NOP.value * AMOUNT_ORDERS_HIGH,
        Action.NOP.value * AMOUNT_ORDERS_HIGH,
        Action.NOP.value * AMOUNT_ORDERS_HIGH,
        Action.NOP.value * AMOUNT_ORDERS_HIGH
    )

for ii, action in enumerate(smart_actions):
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"reward at {ii} step: {reward}")
    total_reward += reward
    done = terminated or truncated
    if done:
        break
print(f"Done: {done}, total reward: {total_reward}")
print(f"Treasury: {env.get_wrapper_attr('mango').treasury_usdc}")
print(f"Health_factor: {env.get_wrapper_attr('mango').get_user_health_factor()}")

reward at 0 step: 0
reward at 1 step: 0
reward at 2 step: 0
reward at 3 step: 10.0
Done: True, total reward: 10.0
Treasury: 0.045268817900039754
Health_factor: -0.08236602430587572
